In [1]:
#!python -m spacy download en_core_web_md
#!pip install swifter

In [33]:
import spacy
import os
import nltk
import string
import pandas as pd
import numpy as np
import warnings
nlp=spacy.load("en_core_web_md")
nltk.download('stopwords')
nltk.download('wordnet')
import re


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
import os
os.chdir('/content/drive/MyDrive/Work/Dan/SVM Model')

In [7]:
from nltk.stem import PorterStemmer
ps=PorterStemmer()
import swifter
stopword=nltk.corpus.stopwords.words('english')
from nltk.stem.wordnet import WordNetLemmatizer

In [56]:
def remove_punct(text):
  text="".join([char for char in text if char not in string.punctuation])
  return text
def tokenization(text):
  text=re.split('\W+',text)
  return text
def remove_stopwords(text):
  text=tokenization(text)
  lemmatizer=WordNetLemmatizer()
  text=[ps.stem(lemmatizer.lemmatize(word)) for word in text if word not in stopword]
  return text

In [62]:
def clean_data(x):
  x=x.encode('ascii','ignore').decode()
  x=x.replace('&nbsp','')
  x=x.replace('&quot','')
  x=x.lower().strip()
  x=re.sub('https*\S+',' ',x)
  x=re.sub('[%s]' % re.escape(string.punctuation),' ',x)
  x=re.sub(r'\W*\d+\W*','',x)
  x=remove_punct(x)
  x=remove_stopwords(x)
  x=list(set(x)-set(['']))
  if len(x)>1:
    x=' '.join(x)
  else:
    x=None
  return x

In [24]:
df_train=pd.read_excel('train_irisdata (1).xlsx')
df_test=pd.read_excel('test_irisdata (1).xlsx')

In [27]:
df_train

utterance                                             intent
0         How to apply for 1Bank account password reset              access_management.reset_1bank_password
1        I am not able to see some application in the...      access_management.missing_application_in_uams
2                              Is bulk request available  access_management.process_bulk_request_bulk_ap...
3                      Why are some application missing?      access_management.missing_application_in_uams
4                        1Bank ID password reset in UAMS             access_management.reset_1bank_password
...                                                  ...                                                ...
13801                                 how to pivot table          microsoft_office_365.create_a_pivot_table
13802                                  how to pivottable          microsoft_office_365.create_a_pivot_table
13803        how to playback recordings from nice portal  voice_services.playback_recording_from_niceportal
13804           how to position my taskbar on my desktop                         computer.misplaced_taskbar
13805           how to prevent machine from hibernating?                           computer.battery_options

[13806 rows x 2 columns]

In [81]:
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score

def train_svm(df,min_df,max_df,ngram_range):
  df.dropna(inplace=True)
  df=df[(df['label']!='nan')&(df['label']!='unknown')]
  df=df[['text_cleaned','label']]
  dd=df.iloc[0].to_dict()
  for i in dd.keys():
    dd[i]=['']
  dd['label']=['Others']
  df=pd.concat([df,pd.DataFrame(dd)])
  vec=TfidfVectorizer(min_df=min_df,ngram_range=ngram_range,max_df=max_df)
  tf=vec.fit(df.text_cleaned)
  X=tf.transform(df.text_cleaned).toarray()
  y=df.label
  tdm_s=pd.DataFrame(X,columns=vec.get_feature_names())
  print('each of the %d utterance is represented by %d features ' %(X.shape))
  sv=LinearSVC()
  sv.fit(X,y)
  sv.intercept_[np.where(sv.classes_=='Others')[0][0]]=max(sv.intercept_)+0.00001
  pred=sv.predict(X)
  print("sv accuracy score ->", accuracy_score(pred,y)*100)
  return sv,tf

In [90]:
def predict_svm(model,tf,dff):
  dff['text_cleaned']=dff['utterance'].dropna().astype(str).apply(lambda x :clean_data(x))
  dff=dff.dropna()
  features_test=tf.transform(dff['text_cleaned'].fillna('')).toarray()
  pred=model.predict(features_test)
  proba=model.decision_function(features_test)
  dff['predictions']=pred
  dff['proba']=np.max(proba,axis=1)
  return dff

In [66]:
df_train['text_cleaned']=df_train['utterance'].swifter.apply(lambda x: clean_data(x))
df_test['text_cleaned']=df_test['utterance'].swifter.apply(lambda x: clean_data(x))

In [82]:
sv,tf=train_svm(df_train.rename(columns={'intent':'label'}),3,0.8,(1,1))

each of the 13597 utterance is represented by 1210 features 
sv accuracy score -> 95.27837022872693


In [91]:
pred=predict_svm(sv,tf,df_test.rename(columns={'intent':'label'}))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [92]:
pred

utterance  ...     proba
0           Add new users to access functional ID  ...  0.660751
1                  Adding new users to access FID  ...  0.665992
2      Apply access for funcional account in UAMS  ... -0.039146
3                      Apply as UAMS coordinator   ...  0.813194
4                      Approver on leave for UAMS  ...  0.888207
...                                           ...  ...       ...
3487                            zoom installation  ...  0.061856
3488                                zoom software  ...  0.228446
3489                                   zoom usage  ...  0.345951
3490                               zoomed browser  ... -0.078996
3491                              zoomed fonts IE  ...  0.956079

[3409 rows x 5 columns]

In [93]:
accuracy_score(pred['label'],pred['predictions'])

0.8363156350836022

In [94]:
pred.to_csv('predictions_iris.csv',index=None)